In [11]:
# Upgrade the library
!conda run -n fpga-env pip install pybondmachine --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: pybondmachine
    Found existing installation: pybondmachine 1.1.73
    Uninstalling pybondmachine-1.1.73:
      Successfully uninstalled pybondmachine-1.1.73



In [1]:
### IF THE PREVIOUS COMMAND DOES NOT RETURN Requirement already satisfied, RELOAD THE KERNEL !!!

In [2]:
import os
import numpy as np

# Export the env variabes to access the alveo card

os.environ["PATH"]=os.environ["PATH"]+":"+os.environ["BONDMACHINE_DIR"]
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2'
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2023.2'
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH']=os.environ["PATH"]+":"+os.environ['XILINX_HLS']+"/bin:"+os.environ['XILINX_VIVADO']+"/bin:"+os.environ['XILINX_VITIS']+"/bin:"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['XILINX_XRT'] = '/opt/xilinx/xrt'
os.environ['LD_LIBRARY_PATH'] = '/opt/xilinx/xrt/lib'

notebook_directory = os.path.abspath(os.path.dirname((os.environ["JPY_SESSION_NAME"])))
os.chdir(notebook_directory)

In [3]:
# generate sample data just
# by default the firmware access a batch of 16 elements of lenght 8
n_samples = 16
length    = 8
np.random.seed(42)
X = np.zeros((n_samples, length), dtype=np.float32)
random_positions = np.random.randint(1, size=n_samples)
X[np.arange(n_samples), random_positions] = 1.0
np.save("features.npy", X)

In [4]:
from pybondmachine.overlay.predictor import Predictor
import numpy as np
model_specs = {
    "data_type": "float32",
    "register_size": 32,
    "batch_size": 16,
    "flavor": "axist",
    "n_input": 8,
    "n_output": 8,
    "benchcore": False,
    "board": "alveo"
}
firmware_name = "bellstate.xclbin"
firmware_path = ""

/opt/conda/envs/fpga-env/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/opt/conda/envs/fpga-env/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [5]:
X_test = np.load("features.npy")
y_test = np.load("features.npy")

In [6]:
predictor = Predictor(firmware_name, firmware_path, model_specs)

In [7]:
# program the ALVEO CARD
predictor.load_overlay()

(True, 'Overlay loaded successfully')

In [8]:
# prepare the data to be sent to the alveo card
predictor.prepare_data(X_test, y_test)


Prepared 1 batches (fill=False, last_batch_size=0)


(True, 'Data prepared successfully')

In [9]:
# run the computation
status, result = predictor.predict(debug=True)

Time taken to predict a batch of size  16  is  0.2665519714355469  ms
Time taken to predict a single sample is  0.01665949821472168  ms
Standard deviation of the time taken to predict a batch of size  16  is  0.0  ms
Standard deviation of the time taken to predict a single sample is  0.0  ms


In [13]:
print(result['all_probs'][0])

[0.7071067690849304, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067690849304, 0.0]
